<a href="https://colab.research.google.com/github/Nata-sanchez/Curso_de_lenguajes_de_programaci-n/blob/main/Institucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy-financial


import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
import time
from IPython.display import clear_output, display

# funcion para mostrar el menú interactivo con botones

In [2]:
url = "https://drive.google.com/uc?id=1rqK5YCbUC87uYIiZeFyScwwJjY8PG1km"

df = pd.read_csv(url, sep=";", encoding="latin1")

In [3]:
# Mostrar la base de datos completa
def MostrarBd():
    display(df)


nombre = ""

# Función para análisis de datos
def Analisis():
    print("Estructura del Dataset:")
    print("-" * 40)
    print("Número de proyectos:", df.shape[0])

    print("\nResumen general:")
    display(df.describe())

    print("\nVerificación de valores nulos:")
    display(df.isnull().sum())

    print("\nProyecto con MAYOR inversión inicial:")
    display(df.loc[df['Inversion_Inicial'] == df['Inversion_Inicial'].min(), ['Proyecto', 'Inversion_Inicial']])

    print("\nProyecto con MENOR inversión inicial:")
    display(df.loc[df['Inversion_Inicial'] == df['Inversion_Inicial'].max(), ['Proyecto', 'Inversion_Inicial']])




In [4]:
def graficar_resultados():
    resultados = []

    for nombre in df['Proyecto']:
        vpn = calcular_vpn_sin_print(nombre)
        tir = calcular_tir_sin_print(nombre)
        if vpn is not None and tir is not None:
            resultados.append({'Proyecto': nombre, 'VPN': vpn, 'TIR': tir})

    resultados_df = pd.DataFrame(resultados)

    # Top 20 proyectos con mayor VPN
    top_vpn = resultados_df.sort_values(by="VPN", ascending=False).head(20)

    plt.figure(figsize=(12, 5))
    plt.barh(top_vpn['Proyecto'], top_vpn['VPN'], color='skyblue')
    plt.xlabel("VPN")
    plt.title("Top 20 proyectos por Valor Presente Neto (VPN)")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.barh(top_vpn['Proyecto'], top_vpn['TIR'] * 100, color='lightgreen')
    plt.axvline(10, color='red', linestyle='--', label='Tasa de descuento (10%)')
    plt.xlabel("TIR (%)")
    plt.title("TIR de los Top 20 proyectos por VPN")
    plt.legend()
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

    # Bottom 20 proyectos con menor VPN
    bottom_vpn = resultados_df.sort_values(by="VPN", ascending=True).head(20)

    plt.figure(figsize=(12, 5))
    plt.barh(bottom_vpn['Proyecto'], bottom_vpn['VPN'], color='salmon')
    plt.xlabel("VPN")
    plt.title("20 proyectos con menor Valor Presente Neto (VPN)")
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

    # Bottom 20 proyectos con menor TIR
    bottom_tir = resultados_df.sort_values(by="TIR", ascending=True).head(20)

    plt.figure(figsize=(12, 5))
    plt.barh(bottom_tir['Proyecto'], bottom_tir['TIR'] * 100, color='orange')
    plt.axvline(10, color='red', linestyle='--', label='Tasa de descuento (10%)')
    plt.xlabel("TIR (%)")
    plt.title("20 proyectos con menor TIR")
    plt.legend()
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()


In [5]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker  # este es el import (No aparezca con notación cientifica)

def graficar_proyecto_individual(nombre):
    fila = df[df['Proyecto'] == nombre]

    if fila.empty:
        print("Proyecto no encontrado.")
        return

    columnas = ['Inversion_Inicial'] + [f'Periodo_{i}' for i in range(1, 11)]
    flujos = fila[columnas].iloc[0].astype(float).values
    periodos = list(range(len(flujos)))

    vpn = npf.npv(0.10, flujos)
    tir = npf.irr(flujos)

    plt.figure(figsize=(10, 5))
    plt.plot(periodos, flujos, marker='o', linestyle='-', color='blue')
    plt.axhline(0, color='gray', linestyle='--')
    plt.title(f"Flujos de caja del proyecto: {nombre}")
    plt.xlabel("Periodo")
    plt.ylabel("Flujo de caja")

    # Formato con separadores de miles(80,000,000)
    plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x:,.0f}'))

    plt.grid(True)
    plt.tight_layout()
    plt.show()

    print(f"VPN: ${vpn:,.2f}")
    print(f"TIR: {tir*100:.2f}%")
    if vpn > 0 and tir > 0.10:
        print("El proyecto es rentable.")
    else:
        print("El proyecto no es rentable.")


In [6]:
def calcular_vpn(nombre_proyecto, tasa_descuento=0.10):

    fila = df[df['Proyecto'] == nombre_proyecto]

    if fila.empty:
        print(f" No se encontró un proyecto con el nombre '{nombre_proyecto}'")
        return None


    # Obtener una sola lista completa: inversión y periodos
    columnas_flujo = ['Inversion_Inicial'] + [f'Periodo_{i}' for i in range(1, 11)]
    flujos = fila[columnas_flujo].iloc[0].astype(float).values  # ya todo en una sola lista

        # Calcular VPN
    vpn = npf.npv(tasa_descuento, flujos)

    print(f" Proyecto: {nombre_proyecto}")
    print(f" VPN = ${vpn:,.2f} con una tasa de descuento del {tasa_descuento*100:.1f}%")
    return vpn



In [7]:
def calcular_vpn_sin_print(nombre, tasa_descuento=0.10):
    fila = df[df['Proyecto'] == nombre]
    if fila.empty:
        return None

    columnas_flujo = ['Inversion_Inicial'] + [f'Periodo_{i}' for i in range(1, 11)]
    flujos = fila[columnas_flujo].iloc[0].astype(float).values
    return npf.npv(tasa_descuento, flujos)

def calcular_tir_sin_print(nombre):
    fila = df[df['Proyecto'] == nombre]
    if fila.empty:
        return None

    columnas_flujo = ['Inversion_Inicial'] + [f'Periodo_{i}' for i in range(1, 11)]
    flujos = fila[columnas_flujo].iloc[0].astype(float).values
    return npf.irr(flujos)


In [8]:
## Calcular TIR

def calcular_tir(nombre):

    fila = df[df['Proyecto'] == nombre]

    if fila.empty:
        print(f" No se encontró un proyecto con el nombre '{nombre}'")
        return None

    columnas_flujo = ['Inversion_Inicial'] + [f'Periodo_{i}' for i in range(1, 11)]
    flujos = fila[columnas_flujo].iloc[0].astype(float).values


    # Calcular la TIR
    tir = npf.irr(flujos)
    if tir>0.10:
      print(f"Proyecto: {nombre} ")
      print(f"TIR = {tir*100:.2f}% \n El proyecto es rentable")
    else :
      print(f"Proyecto: {nombre} ")
      print(f"TIR = {tir*100:.2f}% \n El proyecto no es rentable")

    return tir



In [11]:


def mostrar_menu():

  clear_output() # limpiar la pantalla
  print ("Seleccione una opción:")

  # Crear los botones de cada opción:
  boton_visualizar_datos= widgets.Button(description="Visualizar datos")
  boton_analisis= widgets.Button(description="Analizar datos")
  boton_calcular_VPN_TIR= widgets.Button(description="Calcular VPN Y TIR")
  boton_calcular_VPN= widgets.Button(description="Calcular VPN")
  boton_calcular_TIR= widgets.Button(description="Calcular TIR")
  boton_salir=widgets.Button(description="Salir")
  boton_volver_menu=widgets.Button(description="Volver al menú")
  boton_texto = widgets.Button(description="Enviar")
  boton_texto2 = widgets.Button(description="Enviar")
  Caja_texto = widgets.Text(placeholder="Escriba aquí", description= 'nombre')
  boton_graficar = widgets.Button(description="Graficar resultados")
  boton_graficar_individual = widgets.Button(description="Gráfica de flujos")
  boton_texto_grafica = widgets.Button(description="Mostrar gráfico")
  caja_texto_grafica = widgets.Text(placeholder="Escriba el nombre del proyecto", description='Nombre:')




  #funciones que se ejecutan cuando se hace click en cada boton
  def boton_volver_menu_click(b):
    clear_output()
    display(boton_visualizar_datos, boton_calcular_VPN_TIR, boton_salir)
    mostrar_menu()

  def boton_texto_grafica_click(b):
    clear_output()
    nombre = caja_texto_grafica.value
    graficar_proyecto_individual(nombre)
    display(boton_volver_menu)


  def boton_analisis_click(b):
    clear_output()
    Analisis()
    display(boton_volver_menu)

  def boton_calcular_TIR_click(b):
    clear_output()
    print("usted ha seleccionado el boton de calcular TIR")
    print("Ingrese el nombre del proyecto que desea calcular")
    display(Caja_texto, boton_texto2)
    display(boton_volver_menu)

  def boton_calcular_VPN_click(b):
    clear_output()
    print("usted ha seleccionado el boton de calcular VPN")
    print("Ingrese el nombre del proyecto que desea calcular")
    display(Caja_texto, boton_texto)
    display(boton_volver_menu)


  def boton_visualizar_datos_click(b):
    clear_output()
    print("usted ha seleccionado el boton de visualizar datos")
    MostrarBd()
    display(boton_analisis)
    display(boton_volver_menu)

  def boton_calcular_VPN_TIR_click(b):
    clear_output()
    print("¿Cuál de estas opciones deseas consultar")
    display(boton_calcular_TIR, boton_calcular_VPN)
    display(boton_volver_menu)


  def boton_salir_click(b):
    clear_output()
    print("usted ha seleccionado el boton de salir")

  def boton_texto_click(b):
    clear_output()
    nombre=Caja_texto.value
    calcular_vpn(nombre)
    display(boton_volver_menu)

  def boton_texto_click2(b):
    clear_output()
    nombre=Caja_texto.value
    calcular_tir(nombre)
    display(boton_volver_menu)

  def boton_graficar_click(b):
    clear_output()
    graficar_resultados()
    display(boton_volver_menu)

  def boton_graficar_individual_click(b):
    clear_output()
    print("Ingrese el nombre del proyecto para graficar sus flujos de caja:")
    display(caja_texto_grafica, boton_texto_grafica)
    display(boton_volver_menu)








  # Conexión del boton con las función respectiva

  boton_visualizar_datos.on_click(boton_visualizar_datos_click)
  boton_calcular_VPN_TIR.on_click(boton_calcular_VPN_TIR_click)
  boton_calcular_TIR.on_click(boton_calcular_TIR_click)
  boton_calcular_VPN.on_click(boton_calcular_VPN_click)
  boton_salir.on_click(boton_salir_click)
  boton_volver_menu.on_click(boton_volver_menu_click)
  boton_texto.on_click(boton_texto_click)
  boton_texto2.on_click(boton_texto_click2)
  boton_analisis.on_click(boton_analisis_click)
  boton_graficar.on_click(boton_graficar_click)
  boton_graficar_individual.on_click(boton_graficar_individual_click)
  boton_texto_grafica.on_click(boton_texto_grafica_click)




# Mostrar los botones en la pantalla
  display(boton_visualizar_datos, boton_calcular_VPN_TIR, boton_graficar, boton_graficar_individual, boton_salir)



# ejecutar menú

mostrar_menu()



Seleccione una opción:


Button(description='Visualizar datos', style=ButtonStyle())

Button(description='Calcular VPN Y TIR', style=ButtonStyle())

Button(description='Graficar resultados', style=ButtonStyle())

Button(description='Gráfica de flujos', style=ButtonStyle())

Button(description='Salir', style=ButtonStyle())